In [826]:
import PyPDF2 as pypdf
import pandas as pd
import re

``` 
Question No:n
Question Contant?
A.
B.
C.
D.
Answer:
Explanation:
```

In [451]:
def readFile(filePath):
    try:
        with open(filePath, 'rb') as file:
            pdfFile = pypdf.PdfFileReader(file)
            pagesNum = pdfFile.numPages
            pagesContect=''
            print('reading file please wait...')
            time.sleep(2)
            
            for num in range(pagesNum):
                pagesContect+=pdfFile.getPage(pageNumber=num).extract_text().replace('\n', '')
            else : print('Successfully read PDF file..')
            return pagesContect
    except:
        print('enter corret path of pdf file')

In [452]:
def extractQuestions(fileContent):
    qusetionNum = re.findall('Question No: ?\d', fileContent)

    questions = dict()

    for i, q in enumerate(qusetionNum):
        if q != qusetionNum[-1]:
            x = re.findall(f'(?=Question No: ?{i+1})(.*)(?= Question No: ?{i+2} )', fileContent)
            questions[i+1] = x
        else:
            x = re.findall(f'(?=Question No: ?{i+1})(.*)(?=.)', fileContent)
            questions[i+1] = x
    else : print('Successfully extraction text from PDF file..')
    return questions

In [818]:
def prepareQuestion(questions):
    answer, explan, question, tempQ = [], [], [], []
    answerA, answerB, answerC, answerD = [],  [], [],  []

    # get answers
    for i, n in enumerate(questions.keys()):

        tempQ.append(str(re.findall(f'(?=\\bQuestion No\: ?{i+1})(.*)(?=\\bA\.)', str(questions[n]))).strip('[]'))
        question.append(re.compile(f'(Question No\: ?{i+1})').sub('', tempQ[i]).strip("'"))
        
        answer.append(re.findall(f'(?<=Answer\:)(.*)(?=Explanation\:)', str(questions[n])))
        explan.append(re.findall(f'(?<=\\bExplanation\:)(.*)', str(questions[n])))
        answerA.append(re.findall(f'(?<=\\bA\.)(.*)(?=\\bB\.)', str(questions[n])))
        answerB.append(re.findall(f'(?<=\\bB\.)(.*)(?=\\bC\.)', str(questions[n])))
        answerC.append(re.findall(f'(?<=\\bC\.)(.*)(?=\\bD\.)', str(questions[n])))
        answerD.append(re.findall(f'(?<=\\bD\.)(.*)(?=\\bAnswer\:)', str(questions[n])))
        
    return question, answer, explan, answerA, answerB, answerC, answerD

In [813]:
fileContant = readFile(r'myPdf.pdf')

reading file please wait...
Successfully read PDF file..


In [814]:
extactContant = extractQuestions(fileContant)

Successfully extraction text from PDF file..


In [819]:
question, answer, explan, answerA, answerB, answerC, answerD = prepareQuestion(extactContant)

In [823]:
dit = dict(
    {
        # 'Question':[item for sublist in question for item in sublist],
        'Question':question,
        'answerA': [item for sublist in answerA for item in sublist],
        'answerB':[item for sublist in answerB for item in sublist],
        'answerC':[item for sublist in answerC for item in sublist],
        'answerD':[item for sublist in answerD for item in sublist],
        'Answer': [item for sublist in answer for item in sublist],
        'Explanation': [item for sublist in explan for item in sublist],
    }
)

In [825]:
pd.DataFrame(dit).to_csv('output.csv')

,Question,answerA,answerB,answerC,answerD,Answer,Explanation
0,You are the cluster administrator for your co...,"For the Cluster group, create 10IP Address re...","For Disk group1, create oneIP Address resourc...","For Disk group1, create oneIP Address resourc...","For the Cluster group, create 10IP Address re...",B,To ensurethat users can still use their old s...
1,You are the administrator of aWindows 2000 Ad...,Use Windows Explorer to reassign share-levelp...,Use the cluster administrator to reassign sha...,Use dynamic file shares to remove and then re...,In the Advanced FileShare Properties dialogbox...,B,In order for users to access the shares after...
2,You are the administrator of anew Windows 200...,Add a Network Name resource to the Services R...,Add one Generic Service resource to the Servi...,Add a second IP Address resource to the Servi...,Add two Generic Service resources to the Serv...,A,Wheneveryou create a resource ona cluster you...
